## Esse notebook exemplifica o funcionamento da ferramenta de renderização de mapas inclusa na Toolbox do GT-607.
O algoritmo recebe arquivos .csv e usa das informaçoes de latitude e longitude nos arquivos para renderizar pontos em um mapa. Os pontos podem receber um gradiente de cores baseado em colunas dos arquivos .csv e podem ser filtrados baseados na sua data.
Nesse notebook os pontos gerados recebem como parâmetro os algoritmos do artigo Castro 2020.

In [2]:
import pandas as pd
import keplergl
import configparser
import ast
import glob

In [3]:
global df, kivu, kab_1, sabi, ndci, ndvi, bda_1, bda_2, bda_31, gb1, gr, kepler_map

### Define-se as colunas a serem utilizadas como parâmetros para o gradiente de cores dos pontos.
A função recebe como parâmetro dois arquivos .csv, um com os metadados de fotos de drone contendo as informações de geoposição e data de captura e o outro contendo a mediana extraida de 30% da foto processada em relação ao seu centro.

In [4]:
def map_builder(metadata, chl_algo):
    global df, kivu, kab_1, sabi, ndci, ndvi, bda_1, bda_2, bda_31, bda_mod , gb1,b3b1 , gr

    metadata = pd.read_csv(metadata)
    
    chl_algo = pd.read_csv(chl_algo)

    kivu = pd.concat([metadata, chl_algo['KIVU']], axis=1)

    kab_1 = pd.concat([metadata, chl_algo['Kab 1(Rs)']], axis = 1)

    sabi = pd.concat([metadata, chl_algo['SABI']], axis = 1)

    ndci = pd.concat([metadata, chl_algo['NDCI']], axis = 1)
    
    ndvi = pd.concat([metadata, chl_algo['NDVI']], axis = 1)
    
    bda_1 = pd.concat([metadata, chl_algo['2BDA_1']], axis = 1)
    
    bda_2 = pd.concat([metadata, chl_algo['2BDA_2']], axis = 1)

    bda_31 = pd.concat([metadata, chl_algo['3BDA_1']], axis = 1)
    
    bda_mod = pd.concat([metadata, chl_algo['3BDA_MOD']], axis = 1)
    
    b3b1 = pd.concat([metadata, chl_algo['B3B1']], axis = 1)
    
    gb1 = pd.concat([metadata, chl_algo['GB1']], axis = 1)
    
    gr = pd.concat([metadata, chl_algo['GR']], axis = 1)
    
def df_builder():
    global df
    df = pd.DataFrame()
    df = pd.concat([kivu, sabi, ndci, ndvi, bda_1, bda_2, bda_31, bda_mod, b3b1, gb1, gr], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df

### Adicionando dados ao mapa
A função abaixo permite explicitar quais algoritmos serão adicionados ao mapa. Como mencionado acima, o mapa aceita dados .csv, .xlsx, dataframes e geojsons. 
A função recebe como parâmetro os dataframes que serão adicionados ao mapa.

In [5]:
def add_data_map(kivu, kivu_name, sabi, sabi_name , ndci, ndci_name , ndvi,ndvi_name, bda_1, bda_1_name, bda_2,bda_2_name , bda_31,bda_31_name, bda_mod,bda_mod_name, b3b1,b3b1_name, gb1,gb1_name, gr, gr_name):
    kepler_map.add_data(data=kivu, name=kivu_name)
    kepler_map.add_data(data= sabi, name= sabi_name)
    kepler_map.add_data(data= ndci, name= ndci_name)
    kepler_map.add_data(data= ndvi, name= ndvi_name)
    kepler_map.add_data(data= bda_1, name= bda_1_name)
    kepler_map.add_data(data= bda_2, name= bda_2_name)
    kepler_map.add_data(data= bda_31, name= bda_31_name)
    kepler_map.add_data(data= bda_mod, name= bda_mod_name)
    kepler_map.add_data(data= b3b1, name= b3b1_name)
    kepler_map.add_data(data= gb1, name= gb1_name)
    kepler_map.add_data(data= gr, name= gr_name)

### Renderização do mapa

O mapa abaixo, além de ser renderizado em tempo real em um jupyter-notebook, também é exportado como arquivo .html interativo. 

In [6]:
def fields():
    global kepler_map
    config = configparser.ConfigParser()
    config.read_file(open(r'map_config.txt'))
    map_config = config.get('map_config', 'map_config')
    map_config = ast.literal_eval(map_config)
    
    kepler_map = keplergl.KeplerGl(height=1000, config= map_config)    
    field_data = glob.glob('field_data_*')
    print(field_data)
    df_list = []
    for i in field_data:
        print(i)
        chl_algo = i + '/chl_algo_reflectance_60.csv'
        metadata = i + '/metadata.csv'
        map_builder(metadata, chl_algo)
        data = df_builder()
        df_list.append(data)
    all_data = pd.concat(df_list)
    all_data['DateTime'] = pd.to_datetime(all_data['DateTime'], format = '%Y:%m:%d %H:%M:%S')
    all_data["DateTime"] =  all_data["DateTime"].astype(str)
    add_data_map(all_data,'KIVU', all_data,'SABI' , all_data,'NDCI', all_data,'NDVI', all_data,'2BDA_1', all_data,'2BDA_2', all_data,'3BDA_1', all_data,'3BDA_MOD', all_data,'B3B1', all_data,'GB1', all_data, 'GR')
    kepler_map.save_to_html(file_name = 'map.html')
    kepler_map
fields()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
[]


ValueError: No objects to concatenate